In [36]:
import requests
import pandas as pd
import numpy as np
import json

In [37]:
#Query all data from a year
#Add counter for the number of extractions
data=[]
for i in range(1,30):
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2016","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
data
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


['{"get":"players","parameters":{"league":"39","page":"1","season":"2017"},"errors":[],"results":20,"paging":{"current":1,"total":36},"response":[{"player":{"id":17766,"name":"Luke Coddington","firstname":"Luke","lastname":"Coddington","age":26,"birth":{"date":"1995-06-06","place":"Middlesbrough","country":"England"},"nationality":"England","height":"181 cm","weight":"83 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/17766.png"},"statistics":[{"team":{"id":37,"name":"Huddersfield","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/37.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2017},"games":{"appearences":0,"lineups":0,"minutes":0,"number":null,"position":"Goalkeeper","rating":null,"captain":false},"substitutes":{"in":0,"out":0,"bench":0},"shots":{"total":null,"on":null},"goals":{

In [ ]:
#SLICE DATA THAT WASNT CAPTURED

In [38]:
#complete year array
for i in range(30,37):
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2016","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)
    
data

['{"get":"players","parameters":{"league":"39","page":"1","season":"2017"},"errors":[],"results":20,"paging":{"current":1,"total":36},"response":[{"player":{"id":17766,"name":"Luke Coddington","firstname":"Luke","lastname":"Coddington","age":26,"birth":{"date":"1995-06-06","place":"Middlesbrough","country":"England"},"nationality":"England","height":"181 cm","weight":"83 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/17766.png"},"statistics":[{"team":{"id":37,"name":"Huddersfield","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/37.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2017},"games":{"appearences":0,"lineups":0,"minutes":0,"number":null,"position":"Goalkeeper","rating":null,"captain":false},"substitutes":{"in":0,"out":0,"bench":0},"shots":{"total":null,"on":null},"goals":{

In [56]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

36

In [57]:
len(jsonData[35]['response'])

3

In [58]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [ ]:
#finalPageItems
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])

In [44]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
for i in range(pagesNum):
    
    #To adjust for the last page
    if i == pagesNum-1: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            print(i,j,pkey)
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

0 0 id
0 0 firstname
0 0 lastname
0 0 age
0 0 nationality
0 0 height
0 0 weight
0 0 injured
0 1 id
0 1 firstname
0 1 lastname
0 1 age
0 1 nationality
0 1 height
0 1 weight
0 1 injured
0 2 id
0 2 firstname
0 2 lastname
0 2 age
0 2 nationality
0 2 height
0 2 weight
0 2 injured
0 3 id
0 3 firstname
0 3 lastname
0 3 age
0 3 nationality
0 3 height
0 3 weight
0 3 injured
0 4 id
0 4 firstname
0 4 lastname
0 4 age
0 4 nationality
0 4 height
0 4 weight
0 4 injured
0 5 id
0 5 firstname
0 5 lastname
0 5 age
0 5 nationality
0 5 height
0 5 weight
0 5 injured
0 6 id
0 6 firstname
0 6 lastname
0 6 age
0 6 nationality
0 6 height
0 6 weight
0 6 injured
0 7 id
0 7 firstname
0 7 lastname
0 7 age
0 7 nationality
0 7 height
0 7 weight
0 7 injured
0 8 id
0 8 firstname
0 8 lastname
0 8 age
0 8 nationality
0 8 height
0 8 weight
0 8 injured
0 9 id
0 9 firstname
0 9 lastname
0 9 age
0 9 nationality
0 9 height
0 9 weight
0 9 injured
0 10 id
0 10 firstname
0 10 lastname
0 10 age
0 10 nationality
0 10 height
0 10 

4 14 height
4 14 weight
4 14 injured
4 15 id
4 15 firstname
4 15 lastname
4 15 age
4 15 nationality
4 15 height
4 15 weight
4 15 injured
4 16 id
4 16 firstname
4 16 lastname
4 16 age
4 16 nationality
4 16 height
4 16 weight
4 16 injured
4 17 id
4 17 firstname
4 17 lastname
4 17 age
4 17 nationality
4 17 height
4 17 weight
4 17 injured
4 18 id
4 18 firstname
4 18 lastname
4 18 age
4 18 nationality
4 18 height
4 18 weight
4 18 injured
4 19 id
4 19 firstname
4 19 lastname
4 19 age
4 19 nationality
4 19 height
4 19 weight
4 19 injured
5 0 id
5 0 firstname
5 0 lastname
5 0 age
5 0 nationality
5 0 height
5 0 weight
5 0 injured
5 1 id
5 1 firstname
5 1 lastname
5 1 age
5 1 nationality
5 1 height
5 1 weight
5 1 injured
5 2 id
5 2 firstname
5 2 lastname
5 2 age
5 2 nationality
5 2 height
5 2 weight
5 2 injured
5 3 id
5 3 firstname
5 3 lastname
5 3 age
5 3 nationality
5 3 height
5 3 weight
5 3 injured
5 4 id
5 4 firstname
5 4 lastname
5 4 age
5 4 nationality
5 4 height
5 4 weight
5 4 injured
5 5

9 8 nationality
9 8 height
9 8 weight
9 8 injured
9 9 id
9 9 firstname
9 9 lastname
9 9 age
9 9 nationality
9 9 height
9 9 weight
9 9 injured
9 10 id
9 10 firstname
9 10 lastname
9 10 age
9 10 nationality
9 10 height
9 10 weight
9 10 injured
9 11 id
9 11 firstname
9 11 lastname
9 11 age
9 11 nationality
9 11 height
9 11 weight
9 11 injured
9 12 id
9 12 firstname
9 12 lastname
9 12 age
9 12 nationality
9 12 height
9 12 weight
9 12 injured
9 13 id
9 13 firstname
9 13 lastname
9 13 age
9 13 nationality
9 13 height
9 13 weight
9 13 injured
9 14 id
9 14 firstname
9 14 lastname
9 14 age
9 14 nationality
9 14 height
9 14 weight
9 14 injured
9 15 id
9 15 firstname
9 15 lastname
9 15 age
9 15 nationality
9 15 height
9 15 weight
9 15 injured
9 16 id
9 16 firstname
9 16 lastname
9 16 age
9 16 nationality
9 16 height
9 16 weight
9 16 injured
9 17 id
9 17 firstname
9 17 lastname
9 17 age
9 17 nationality
9 17 height
9 17 weight
9 17 injured
9 18 id
9 18 firstname
9 18 lastname
9 18 age
9 18 nationa

13 8 age
13 8 nationality
13 8 height
13 8 weight
13 8 injured
13 9 id
13 9 firstname
13 9 lastname
13 9 age
13 9 nationality
13 9 height
13 9 weight
13 9 injured
13 10 id
13 10 firstname
13 10 lastname
13 10 age
13 10 nationality
13 10 height
13 10 weight
13 10 injured
13 11 id
13 11 firstname
13 11 lastname
13 11 age
13 11 nationality
13 11 height
13 11 weight
13 11 injured
13 12 id
13 12 firstname
13 12 lastname
13 12 age
13 12 nationality
13 12 height
13 12 weight
13 12 injured
13 13 id
13 13 firstname
13 13 lastname
13 13 age
13 13 nationality
13 13 height
13 13 weight
13 13 injured
13 14 id
13 14 firstname
13 14 lastname
13 14 age
13 14 nationality
13 14 height
13 14 weight
13 14 injured
13 15 id
13 15 firstname
13 15 lastname
13 15 age
13 15 nationality
13 15 height
13 15 weight
13 15 injured
13 16 id
13 16 firstname
13 16 lastname
13 16 age
13 16 nationality
13 16 height
13 16 weight
13 16 injured
13 17 id
13 17 firstname
13 17 lastname
13 17 age
13 17 nationality
13 17 height


17 9 firstname
17 9 lastname
17 9 age
17 9 nationality
17 9 height
17 9 weight
17 9 injured
17 10 id
17 10 firstname
17 10 lastname
17 10 age
17 10 nationality
17 10 height
17 10 weight
17 10 injured
17 11 id
17 11 firstname
17 11 lastname
17 11 age
17 11 nationality
17 11 height
17 11 weight
17 11 injured
17 12 id
17 12 firstname
17 12 lastname
17 12 age
17 12 nationality
17 12 height
17 12 weight
17 12 injured
17 13 id
17 13 firstname
17 13 lastname
17 13 age
17 13 nationality
17 13 height
17 13 weight
17 13 injured
17 14 id
17 14 firstname
17 14 lastname
17 14 age
17 14 nationality
17 14 height
17 14 weight
17 14 injured
17 15 id
17 15 firstname
17 15 lastname
17 15 age
17 15 nationality
17 15 height
17 15 weight
17 15 injured
17 16 id
17 16 firstname
17 16 lastname
17 16 age
17 16 nationality
17 16 height
17 16 weight
17 16 injured
17 17 id
17 17 firstname
17 17 lastname
17 17 age
17 17 nationality
17 17 height
17 17 weight
17 17 injured
17 18 id
17 18 firstname
17 18 lastname
17 1

22 3 injured
22 4 id
22 4 firstname
22 4 lastname
22 4 age
22 4 nationality
22 4 height
22 4 weight
22 4 injured
22 5 id
22 5 firstname
22 5 lastname
22 5 age
22 5 nationality
22 5 height
22 5 weight
22 5 injured
22 6 id
22 6 firstname
22 6 lastname
22 6 age
22 6 nationality
22 6 height
22 6 weight
22 6 injured
22 7 id
22 7 firstname
22 7 lastname
22 7 age
22 7 nationality
22 7 height
22 7 weight
22 7 injured
22 8 id
22 8 firstname
22 8 lastname
22 8 age
22 8 nationality
22 8 height
22 8 weight
22 8 injured
22 9 id
22 9 firstname
22 9 lastname
22 9 age
22 9 nationality
22 9 height
22 9 weight
22 9 injured
22 10 id
22 10 firstname
22 10 lastname
22 10 age
22 10 nationality
22 10 height
22 10 weight
22 10 injured
22 11 id
22 11 firstname
22 11 lastname
22 11 age
22 11 nationality
22 11 height
22 11 weight
22 11 injured
22 12 id
22 12 firstname
22 12 lastname
22 12 age
22 12 nationality
22 12 height
22 12 weight
22 12 injured
22 13 id
22 13 firstname
22 13 lastname
22 13 age
22 13 nationa

26 4 age
26 4 nationality
26 4 height
26 4 weight
26 4 injured
26 5 id
26 5 firstname
26 5 lastname
26 5 age
26 5 nationality
26 5 height
26 5 weight
26 5 injured
26 6 id
26 6 firstname
26 6 lastname
26 6 age
26 6 nationality
26 6 height
26 6 weight
26 6 injured
26 7 id
26 7 firstname
26 7 lastname
26 7 age
26 7 nationality
26 7 height
26 7 weight
26 7 injured
26 8 id
26 8 firstname
26 8 lastname
26 8 age
26 8 nationality
26 8 height
26 8 weight
26 8 injured
26 9 id
26 9 firstname
26 9 lastname
26 9 age
26 9 nationality
26 9 height
26 9 weight
26 9 injured
26 10 id
26 10 firstname
26 10 lastname
26 10 age
26 10 nationality
26 10 height
26 10 weight
26 10 injured
26 11 id
26 11 firstname
26 11 lastname
26 11 age
26 11 nationality
26 11 height
26 11 weight
26 11 injured
26 12 id
26 12 firstname
26 12 lastname
26 12 age
26 12 nationality
26 12 height
26 12 weight
26 12 injured
26 13 id
26 13 firstname
26 13 lastname
26 13 age
26 13 nationality
26 13 height
26 13 weight
26 13 injured
26 14

30 4 weight
30 4 injured
30 5 id
30 5 firstname
30 5 lastname
30 5 age
30 5 nationality
30 5 height
30 5 weight
30 5 injured
30 6 id
30 6 firstname
30 6 lastname
30 6 age
30 6 nationality
30 6 height
30 6 weight
30 6 injured
30 7 id
30 7 firstname
30 7 lastname
30 7 age
30 7 nationality
30 7 height
30 7 weight
30 7 injured
30 8 id
30 8 firstname
30 8 lastname
30 8 age
30 8 nationality
30 8 height
30 8 weight
30 8 injured
30 9 id
30 9 firstname
30 9 lastname
30 9 age
30 9 nationality
30 9 height
30 9 weight
30 9 injured
30 10 id
30 10 firstname
30 10 lastname
30 10 age
30 10 nationality
30 10 height
30 10 weight
30 10 injured
30 11 id
30 11 firstname
30 11 lastname
30 11 age
30 11 nationality
30 11 height
30 11 weight
30 11 injured
30 12 id
30 12 firstname
30 12 lastname
30 12 age
30 12 nationality
30 12 height
30 12 weight
30 12 injured
30 13 id
30 13 firstname
30 13 lastname
30 13 age
30 13 nationality
30 13 height
30 13 weight
30 13 injured
30 14 id
30 14 firstname
30 14 lastname
30 

34 6 id
34 6 firstname
34 6 lastname
34 6 age
34 6 nationality
34 6 height
34 6 weight
34 6 injured
34 7 id
34 7 firstname
34 7 lastname
34 7 age
34 7 nationality
34 7 height
34 7 weight
34 7 injured
34 8 id
34 8 firstname
34 8 lastname
34 8 age
34 8 nationality
34 8 height
34 8 weight
34 8 injured
34 9 id
34 9 firstname
34 9 lastname
34 9 age
34 9 nationality
34 9 height
34 9 weight
34 9 injured
34 10 id
34 10 firstname
34 10 lastname
34 10 age
34 10 nationality
34 10 height
34 10 weight
34 10 injured
34 11 id
34 11 firstname
34 11 lastname
34 11 age
34 11 nationality
34 11 height
34 11 weight
34 11 injured
34 12 id
34 12 firstname
34 12 lastname
34 12 age
34 12 nationality
34 12 height
34 12 weight
34 12 injured
34 13 id
34 13 firstname
34 13 lastname
34 13 age
34 13 nationality
34 13 height
34 13 weight
34 13 injured
34 14 id
34 14 firstname
34 14 lastname
34 14 age
34 14 nationality
34 14 height
34 14 weight
34 14 injured
34 15 id
34 15 firstname
34 15 lastname
34 15 age
34 15 nati

In [45]:
len(finalArr)
#len(proto)

703

In [46]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[17766, 'Luke', 'Coddington', ..., None, None, None],
       [18585, 'Dan', 'Woodards', ..., None, None, None],
       [18593, 'Ricky', 'Shakes', ..., None, None, None],
       ...,
       [19236, 'Muhamed', 'Bešić', ..., 0, 0, None],
       [37154, 'Jordy', 'Clasie', ..., None, None, None],
       [44827, 'Ádám', 'Bogdán', ..., None, None, None]], dtype=object)

In [47]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,17766,Luke,Coddington,26,England,181 cm,83 kg,False,37,Huddersfield,...,None,None,0,0,0,None,None,None,None,None
1,18585,Dan,Woodards,38,England,180 cm,80 kg,False,1830,Boreham Wood,...,None,None,0,0,0,None,None,None,None,None
2,18593,Ricky,Shakes,36,Guyana,178 cm,76 kg,False,1830,Boreham Wood,...,None,None,0,0,0,None,None,None,None,None
3,18774,Wes,Morgan,37,Jamaica,186 cm,101 kg,False,46,Leicester,...,12,31,4,0,0,None,None,0,0,None
4,18879,Andrew,Surman,35,England,178 cm,73 kg,False,35,Bournemouth,...,14,9,2,0,0,None,None,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,30531,Jérémie,Boga,25,Côte d'Ivoire,172 cm,68 kg,False,49,Chelsea,...,0,1,0,0,0,None,None,0,0,None
699,47181,Antonio,Martínez López,25,Spain,180 cm,70 kg,False,48,West Ham,...,None,None,0,0,0,None,None,None,None,None
700,19236,Muhamed,Bešić,30,Bosnia and Herzegovina,180 cm,80 kg,False,45,Everton,...,2,1,0,0,0,None,None,0,0,None
701,37154,Jordy,Clasie,31,Netherlands,169 cm,68 kg,False,41,Southampton,...,None,None,0,0,0,None,None,None,None,None


In [48]:
gb = df.groupby(df['lastname']).count()#=='Kane'].goalAssists
df[df['lastname']=='Morgan']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
3,18774,Wes,Morgan,37,Jamaica,186 cm,101 kg,False,46,Leicester,...,12,31,4,0,0,None,None,0,0,None


In [49]:
df['season'] = '2016/2017'

In [50]:
#Export the df to a csv
df.to_csv('data2016&2017.csv')